In [ ]:
# First time
%cd /content
!git clone https://github.com/FireMight/point-cloud-retrieval-from-image.git
%cd /content/point-cloud-retrieval-from-image
!git submodule update --init --recursive
%cd ..

In [ ]:
# Pull repository
%cd /content/point-cloud-retrieval-from-image
!git pull --recurse-submodules
%cd ..

In [2]:
import csv
import os
import sys
import math
import numpy as np
from datetime import datetime
from PIL import Image

In [ ]:
# For Google colab
from google.colab import drive
drive.mount('/content/drive')

sdk_dir = '/content/point-cloud-retrieval-from-image/data/oxford/robotcar-dataset-sdk'
project_dir = '/content/drive/My Drive/ADL4CV'

run_id = 'reference'
data_dir = project_dir + '/downloads/oxford_dataset/' + run_id
gps_dir = data_dir + '/gps'
ins_data_file = gps_dir + '/ins.csv'
extrinsics_dir = sdk_dir + '/extrinsics'

In [3]:
# For local use
sdk_dir = 'data/oxford/robotcar-dataset-sdk'

run_id = 'reference'
data_dir = 'data/oxford/data/'  + run_id
gps_dir = data_dir + '/gps'
ins_data_file = gps_dir + '/ins.csv'
extrinsics_dir = sdk_dir + '/extrinsics'

In [10]:
def read_metadata_csv_row(row):
    metadata = {}
    metadata['seg_idx'] = int(row['seg_idx'])
    metadata['timestamp_start'] = int(row['timestamp_start'])
    metadata['northing_start'] = float(row['northing_start'])
    metadata['easting_start'] = float(row['easting_start'])
    metadata['down_start'] = float(row['down_start'])
    metadata['heading_start'] = float(row['heading_start'])
    metadata['timestamp_center'] = int(row['timestamp_center'])
    metadata['northing_center'] = float(row['northing_center'])
    metadata['easting_center'] = float(row['easting_center'])
    metadata['down_center'] = float(row['down_center'])
    metadata['heading_center'] = float(row['heading_center'])
    
    return metadata


def get_pcl_metadata(metadata_file, seg_idx=None):
    if seg_idx is None:
        # Return list of all metadata
        metadata = []
    else:
        # Return metadata of single submap
        metadata = None
        
    with open(metadata_file) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if seg_idx is None:
                metadata.append(read_metadata_csv_row(row))
            elif int(row['seg_idx']) == seg_idx:
                metadata = read_metadata_csv_row(row)
                break
    return metadata

In [51]:
sys.path.insert(0, sdk_dir + '/python')
import image as sdk_image
from camera_model import CameraModel

class ImageLoader():
    def __init__(self, camera_type, center_offset=None):
        self.trajectory_ned = None
        self.subset_split = None
        self.camera_type = camera_type
        self.center_offset = center_offset
        
        if camera_type == 'center':
            self.img_dir = data_dir + '/stereo/centre'
            self.timestamps_file = data_dir + '/stereo.timestamps'
            self.tgt_dir = data_dir + '/img_center'
            if center_offset is not None:
                self.tgt_dir += '_{}'.format(int(center_offset))
        elif camera_type in ['left', 'right']:
            self.img_dir = data_dir + '/mono_{}'.format(camera_type)
            self.timestamps_file = data_dir + '/mono_{}.timestamps'.format(camera_type)
            self.tgt_dir = data_dir + '/img_{}'.format(camera_type)
        else:
            raise ValueError('Wrong camera type: ', camera_type)
        
        if not os.path.isdir(self.tgt_dir):
            os.mkdir(self.tgt_dir)
            
        self.camera_model = CameraModel(sdk_dir + '/models', self.img_dir)
        
    
    def load_ned_trajectory(self, subset_split=None):
        # Get timestamps from camera data
        self.timestamps_file = np.genfromtxt(self.timestamps_file)
        self.timestamps_file = self.timestamps_file[:,0].flatten()
        start_time = self.timestamps_file[0]
        end_time = self.timestamps_file[-1]

        # Check if NED trajectory has already been computed
        trajectory_file = gps_dir + '/ned_trajectory_{}_{}.npy'.format(self.camera_type, 
                                                                       run_id)
        print('Search for precomputed NED trajectory ' + trajectory_file + '...')

        try:
            self.trajectory_ned = np.load(trajectory_file)
            print('Done!')
        except:
            print('Not found! Create camera trajectory...')
            # Get trajectory corresponding to camera data
            camera_timestamp_idx = 0
            self.trajectory_ned = np.empty((7,0))
            with open(ins_data_file, 'r') as ins_file:
                reader = csv.DictReader(ins_file)
                for row in reader:
                    if int(row['timestamp']) > end_time:
                        break
                    if int(row['timestamp']) < start_time:
                        continue

                    # Get closest corresponding camera timestamp
                    ins_timestamp = int(row['timestamp'])
                    camera_timestamp_idx = self.get_closest_camera_timestamp(
                                                ins_timestamp, camera_timestamp_idx)
                    img_timestamp = self.timestamps_file[camera_timestamp_idx]

                    camera_state = np.array([float(row['northing']),
                                             float(row['easting']),
                                             float(row['down']),
                                             float(row['roll']),
                                             float(row['pitch']),
                                             float(row['yaw']),
                                             img_timestamp]).reshape(7,1)
                    self.trajectory_ned = np.append(self.trajectory_ned, camera_state, axis=1)
 
            print('Done! Trajectory with {} samples'.format(
                                                  self.trajectory_ned.shape[1]))

            # Save trajectory
            print('Save NED trajectory to ' + trajectory_file + '...')
            np.save(trajectory_file, self.trajectory_ned)
            print('Done!')
            
        # If specified, use only a subset of the data
        if subset_split is not None:
            max_idx = int(self.trajectory_ned.shape[1] * subset_split)
            print('WARNING: We use only the first {}% of the trajectory ({} measurements)'.
                                                format(int(subset_split*100), max_idx))
            self.trajectory_ned = self.trajectory_ned[:,:max_idx]
            self.subset_split = subset_split
            
    def load_images(self, submap_length):
        # Load pointcloud metadata
        pcl_dir = data_dir + '/submaps_{}m'.format(int(submap_length))
        pcl_metadata_filename = (pcl_dir + '/metadata.csv')
        pcl_metadata = get_pcl_metadata(pcl_metadata_filename)

        # Sort list by timestamp
        pcl_metadata.sort(key=lambda pcl: pcl['timestamp_start'])

        # Create image metadata csv file
        metadata_fieldnames = ['seg_idx',
                               'camera',
                               'offset', 
                               'timestamp',
                               'northing',
                               'easting',
                               'down',
                               'heading', 
                               'dist_to_ref']
        metadata_csv = (self.tgt_dir + '/metadata.csv')

        with open(metadata_csv, 'w') as outcsv:
            writer = csv.DictWriter(outcsv, metadata_fieldnames)
            writer.writeheader()
        
        # Walk along camera trajectory and save all images belonging to a submap
        if self.camera_type == 'center':
            ref = 'start'
            offset = self.center_offset
        else:
            ref = 'center'
            offset = 0
            
        i_pcl = 0
        i_min = 0
        dist_min = 1.0e9
        ref_pos = np.array([pcl_metadata[i_pcl]['northing_'+ref],
                            pcl_metadata[i_pcl]['easting_'+ref]])
                
        for i in range(self.trajectory_ned.shape[1]):
            pos = self.trajectory_ned[:2,i]
            dist_to_ref = np.linalg.norm(pos - ref_pos)

            if dist_to_ref < dist_min:
                i_min = i
                dist_min = dist_to_ref

            if dist_to_ref > dist_min + 0.5:

                # Go back to offset
                if offset > 0:
                    dist_to_offset = offset
                    prev_pos = self.trajectory_ned[:3,i_min]
                    for i_min in range(i_min-1,-1,-1):
                        curr_pos = self.trajectory_ned[[0,1,2],i_min]
                        dist_to_offset -= np.linalg.norm(prev_pos - curr_pos)
                        prev_pos = curr_pos
                        if dist_to_offset < 0:
                            break

                print('Save image {} with dist {}'.format(i_min, dist_min))
                # Save image of step with minimum dist to reference as png
                timestamp = self.trajectory_ned[6,i_min]
                seg_idx = pcl_metadata[i_pcl]['seg_idx']

                _, pil_image = self.load_image(timestamp, size=(320,200))
                pil_image.save(self.tgt_dir + '/img_{}_{}.png'.format(submap_length, 
                                                                         seg_idx), 'PNG')

                # Save image metadata
                with open(metadata_csv, 'a') as outcsv:
                    writer = csv.DictWriter(outcsv,fieldnames=metadata_fieldnames)
                    writer.writerow({'seg_idx' : seg_idx,
                                     'camera' : self.camera_type,
                                     'offset' : self.center_offset, 
                                     'timestamp' : int(timestamp),
                                     'northing' : self.trajectory_ned[0,i_min],
                                     'easting' : self.trajectory_ned[1,i_min],
                                     'down' : self.trajectory_ned[2,i_min],
                                     'heading' : self.trajectory_ned[5,i_min], 
                                     'dist_to_ref' : dist_min})


                # Set new reference position
                i_pcl += 1
                ref_pos = np.array([pcl_metadata[i_pcl]['northing_'+ref],
                                    pcl_metadata[i_pcl]['easting_'+ref]])
                dist_min = 1.0e9
            
        
    
    def get_closest_camera_timestamp(self, ins_timestamp, start_idx):
        min_diff = 1e9
        i_min = start_idx
        for i in range(start_idx, self.timestamps_file.shape[0]):
            diff = math.fabs(ins_timestamp - self.timestamps_file[i])

            if diff < min_diff:
                min_diff = diff
                i_min = i
            else:
                break

        return i_min
    
    def load_image(self, timestamp, size=None):
        image_file = os.path.join(self.img_dir, '{}.png'.format(int(timestamp)))
        image = sdk_image.load_image(image_file, model=self.camera_model)

        pil_image = Image.fromarray(image.astype('uint8'))
        if size is not None:
            pil_image = pil_image.resize(size)

        return image, pil_image

In [52]:
camera_type = 'right'
offset = None
submap_length = 20

img_loader = ImageLoader(camera_type, center_offset=offset)
img_loader.load_ned_trajectory(subset_split=0.2)
img_loader.load_images(submap_length)

Search for precomputed NED trajectory data/oxford/data/reference/gps/ned_trajectory_right_reference.npy...
Done!
Save image 1005 with dist 0.0
Save image 1034 with dist 0.0
Save image 1053 with dist 0.0
Save image 1065 with dist 0.0
Save image 1079 with dist 0.0
Save image 1092 with dist 0.0
Save image 1106 with dist 0.0
Save image 1118 with dist 0.0
Save image 1131 with dist 0.0
Save image 1143 with dist 0.0
Save image 1155 with dist 0.0
Save image 1167 with dist 0.0
Save image 1179 with dist 0.0
Save image 1191 with dist 0.0
Save image 1203 with dist 0.0
Save image 1215 with dist 0.0
Save image 1228 with dist 0.0
Save image 1241 with dist 0.0
Save image 1254 with dist 0.0
Save image 1267 with dist 0.0
Save image 1281 with dist 0.0
Save image 1295 with dist 0.0
Save image 1305 with dist 0.0
Save image 1320 with dist 0.0
Save image 1336 with dist 0.0
Save image 1353 with dist 0.0
Save image 1367 with dist 0.0
Save image 1386 with dist 0.0
Save image 1405 with dist 0.0
Save image 1426 w

IndexError: list index out of range